In [ ]:
import numpy
import pandas as pd
import os
import csv
import ast
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

from wquantiles import quantile
from openfisca_survey_manager.utils import asof

from openfisca_france_indirect_taxation import FranceIndirectTaxationTaxBenefitSystem
from openfisca_france_indirect_taxation.examples.utils_example import (
    wavg,
    collapse,
    dataframe_by_group,
    graph_builder_bar,
    df_weighted_average_grouped)
from openfisca_france_indirect_taxation.build_survey_data.utils import weighted_sum
from openfisca_france_indirect_taxation.almost_ideal_demand_system.utils import add_niveau_vie_decile
from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.Calage_consommation_bdf import getinflators
from openfisca_france_indirect_taxation.utils import assets_directory, get_input_data_frame

In [ ]:
from openfisca_france_indirect_taxation.Calage_consommation_bdf import (
    get_cn_aggregates,
    get_inflators_cn_to_cn,
    get_inflators,
    get_inflators_by_year)

In [ ]:
data_year = 2017
inflators_by_year = getinflators(2017, 2025)
simulated_variables = ['tva_taux_plein',
'tva_taux_intermediaire',
 'tva_taux_reduit',
 'tva_taux_super_reduit',
 'tva_total',
 'depenses_tva_exonere',
 'depenses_tva_taux_plein',
 'depenses_tva_taux_intermediaire',
 'depenses_tva_taux_reduit',
 'depenses_tva_taux_super_reduit',
 'rev_disponible',
 'rev_disp_yc_loyerimpute',
 'loyer_impute',
 'depenses_tot',
 'depenses_totales',
 'pondmen',
]

agregates = pd.DataFrame(columns = simulated_variables)
for year in range(2017,2025) :
    inflation_kwargs = dict(inflator_by_variable = inflators_by_year[year])
    survey_scenario = SurveyScenario.create(
        inflation_kwargs = inflation_kwargs,
        year = year,
        data_year = data_year
        )
    df_sum = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'sum')
    agregates.loc[year] = df_sum.sum()

In [ ]:
agregates['Conso BDF'] = agregates['depenses_tot'] + agregates['loyer_impute']
agregates.reset_index(inplace= True)
agregates.rename(columns = {'index' : 'year'}, inplace= True)
agregates

In [ ]:
cn_agregates_by_year = pd.DataFrame()
for year in range(2017,2024):
    cn_agregates = getcn_aggregates(year)
    cn_agregates.drop(['poste_02_4', 'poste_04_4_3','rev_disp_yc_loyerimpute','rev_disponible'], axis= 0, inplace = True)
    cn_agregates_by_year = pd.concat([cn_agregates_by_year,pd.DataFrame(cn_agregates.sum())])
cn_agregates_by_year
cn_agregates_by_year.reset_index(inplace = True)
cn_agregates_by_year['year'] = cn_agregates_by_year['index'].apply(lambda x : x[-4:]).astype(int)
cn_agregates_by_year.drop('index', axis = 1, inplace= True)
cn_agregates_by_year.rename(columns = {0 : 'Conso CN'}, inplace= True)
cn_agregates_by_year

In [ ]:
to_compare = agregates.loc[:,['Conso BDF']].merge(cn_agregates_by_year, left_index= True, right_index= True)
to_compare.set_index('year', inplace = True)
to_compare = to_compare* 1e-9
to_compare = to_compare.round(1)

In [ ]:
to_compare['Ecart BDF / CN'] = (to_compare['Conso CN'] / to_compare['Conso BDF'] -1)* 100
to_compare

In [ ]:
to_compare['Growth_rate_CN'] = to_compare['Conso CN'].pct_change()
to_compare

**Par postes agrégés**

In [ ]:
from openfisca_france_indirect_taxation.projects.TVA.new_calage_bdf_cn_by_postes_agreges import (get_bdf_aggregates,
    get_cn_aggregates,
    get_inflators_bdf_to_cn,
    get_inflators_cn_to_cn,
    get_inflators_by_year)

In [ ]:
aggregates_13_postes = get_cn_aggregates(2023)
aggregates_13_postes


In [ ]:
cn_agregates = getcn_aggregates(2023)
to_substract = cn_agregates.loc[cn_agregates.index.isin(['poste_02_4', 'poste_04_4_3','loyer_impute'])]
to_substract.rename(index ={'loyer_impute' : 'poste_04_2'}, columns = {'conso_CN_2023' : 'to_substract'}, inplace = True)
to_substract = to_substract.groupby(to_substract.index.str[:8]).sum()

In [ ]:
aggregates_13_postes = aggregates_13_postes.merge(to_substract, left_index= True, right_index= True, how = 'left')
aggregates_13_postes.fillna(0, inplace = True)

In [ ]:
aggregates_13_postes['right_conso_CN_2023'] = aggregates_13_postes['conso_CN_2023'] - aggregates_13_postes['to_substract']
aggregates_13_postes.drop(columns=['conso_CN_2023','to_substract'], axis= 1,  inplace= True)
aggregates_13_postes.rename(columns={'right_conso_CN_2023' : 'conso_CN_2023'}, inplace = True)

aggregates_13_postes.index = aggregates_13_postes.index.to_series().apply(lambda x :x.replace('poste_' ,'poste_agrege_'))

In [ ]:
data_year = 2017
inflators = getinflators(2023,2017)
simulated_variables = ["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_10", "poste_agrege_11", "poste_agrege_12", "poste_agrege_13"]

postes_agreges = pd.DataFrame(columns=["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_10", "poste_agrege_11", "poste_agrege_12", "poste_agrege_13"])
inflation_kwargs = dict(inflator_by_variable = inflators)
survey_scenario = SurveyScenario.create(
    inflation_kwargs = inflation_kwargs,
    year = 2023,
    data_year = data_year
    )
df_sum = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'sum')
postes_agreges = df_sum.sum()
conso_by_poste = pd.DataFrame(postes_agreges.T).reset_index().rename(columns={'index' : 'Code', 0 : 'conso_OFF_IT_2023'}).set_index('Code')

In [ ]:
aggregates_13_postes = aggregates_13_postes.merge(conso_by_poste, left_index= True, right_index= True)
aggregates_13_postes = aggregates_13_postes * 1e-9
aggregates_13_postes = aggregates_13_postes.round(2)
aggregates_13_postes.rename({'poste_agrege_13' : 'poste_agrege_12' },axis = 0, inplace = True)
aggregates_13_postes = aggregates_13_postes.groupby(aggregates_13_postes.index).sum()

In [ ]:
label_postes_agreges = ['Alimentation', 'Alcools et tabacs', 'Habillement et chaussures', 'Logement', 'Ameublement et équipement ménager', 'Santé',
                        'Transports', 'Communication' , 'Loisirs et cultures' , 'Education', 'Restauration et hôtellerie', 'Services et biens divers']
liste_postes_agreges = ["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_{}".format(i) for i in range(10, 13)]
label_dict = dict(zip(liste_postes_agreges,label_postes_agreges))

In [ ]:
aggregates_13_postes.rename(label_dict, axis = 0, inplace = True)

In [ ]:
aggregates_13_postes